<a href="https://colab.research.google.com/github/parth-mango/EVA5-Assignments/blob/main/EVA_A5F1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Target
  * Setting up a skeleton 
  * Setting a basic Architecture with GAP to remove the final layer


#Result
  * Parameters - 19,734
  * Performance - 98.75%

#Analysis
  * We have to use batchnorm to increase contrast
  * Performance is way below the goal of 99.4


In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
def calc_rf(jin, k, rin):
    rout = rin + ((k-1) * jin)
    return rout

calc_rf(4,1,40)

40

In [ ]:
 #Model paramters: 19942

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1, 4, 3, padding=1),    # 28x28x1 > 28x28x4   : RF 3x3
            nn.ReLU(),
            nn.Conv2d(4, 8, 3, padding=1),    # 28x28x4 > 28x28x8   : RF 5x5
            nn.ReLU(),
            nn.Conv2d(8, 12, 3, padding=1),   # 28x28x8 > 28x28x12  : RF 7x7
            nn.ReLU(),
            nn.Conv2d(12, 16, 3, padding=1),  # 28x28x12 > 28x28x16 : RF 9x9
            nn.ReLU(),
            nn.MaxPool2d(2,2),                # 28x28x16 > 14x14x16 : RF 10x10
            nn.Conv2d(16, 32, 3, padding=1),  # 14x14x16 > 14x14x32 : RF 14x14
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),  # 14x14x32 > 14x14x32 : RF 18x18
            nn.ReLU(),
            nn.MaxPool2d(2,2),                # 14x14x32 > 7x7x32   : RF 20x20
            nn.Conv2d(32, 10, 3),             # 7x7x32 > 5x5x10     : RF 28x28
            nn.AvgPool2d(5, 2),               # 5x5x10 > 1x1x10     : RF 32x32
        )
 
    def forward(self, x):
        x = self.conv(x)
        
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
              ReLU-2            [-1, 4, 28, 28]               0
            Conv2d-3            [-1, 8, 28, 28]             296
              ReLU-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 12, 28, 28]             876
              ReLU-6           [-1, 12, 28, 28]               0
            Conv2d-7           [-1, 16, 28, 28]           1,744
              ReLU-8           [-1, 16, 28, 28]               0
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10           [-1, 32, 14, 14]           4,640
             ReLU-11           [-1, 32, 14, 14]               0
           Conv2d-12           [-1, 32, 14, 14]           9,248
             ReLU-13           [-1, 32, 14, 14]               0
        MaxPool2d-14             [-1, 3

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    y_pred = model(data)
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)
    loss.backward()
    optimizer.step()
    pred = y_pred.argmax(dim=1, keepdim=True)  
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)
    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)
    print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))


Loss=2.304893732070923 Batch_id=468 Accuracy=11.19: 100%|██████████| 469/469 [00:13<00:00, 33.97it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.2944, Accuracy: 1817/10000 (18.17%)

Epoch-1 lr: 0.01


Loss=0.21541088819503784 Batch_id=468 Accuracy=62.41: 100%|██████████| 469/469 [00:14<00:00, 33.38it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2957, Accuracy: 9113/10000 (91.13%)

Epoch-2 lr: 0.01


Loss=0.18518303334712982 Batch_id=468 Accuracy=92.49: 100%|██████████| 469/469 [00:13<00:00, 34.05it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0451, Accuracy: 9868/10000 (98.68%)

Epoch-13 lr: 0.01


Loss=0.009677485562860966 Batch_id=468 Accuracy=98.79: 100%|██████████| 469/469 [00:13<00:00, 34.23it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0350, Accuracy: 9890/10000 (98.90%)

Epoch-14 lr: 0.01


Loss=0.022144312039017677 Batch_id=468 Accuracy=98.80: 100%|██████████| 469/469 [00:13<00:00, 34.87it/s]



Test set: Average loss: 0.0370, Accuracy: 9875/10000 (98.75%)

Epoch-15 lr: 0.01
